<a id='Top'></a>
# Independent Study: Computational Orbital Mechanics#
Joshua Gates

Instructor: Vincent Pisacane

Contents:
1. [Course Information](#CourseInfo)
1. [Weekly Progress](#WeeklyProg)


<a id='CourseInfo'></a>
## Course Information##
[Top](#Top)
### BRIEF COURSE DESCRIPTION ###
An important topic in astrodynamics is the solution of boundary value problems with mixed boundary conditions and optimization of an object function. This directed studies course will focus on learning the techniques involved and applying them to a problem in astrodynamics.

### LEARNING OBJECTIVES###
*	 Understand the fundamentals of selected numerical techniques in the solution of boundary value problems with mixed boundary conditions without and with optimization.
* 	Apply the techniques to problems in astrodynamics.

### COURSE STRUCTURE AND LEARNING RESOURCES TO BE USED ###
1. 	Document the fundamentals of the single shooting technique for boundary value problems with mixed boundary condition without and with optimization. Demonstrate proficiency by application to solution of specific examples. 
2.  	Document the fundamentals of the multiple shooting technique for boundary value problems with mixed boundary condition without and with optimization. Demonstrate proficiency by application to the solution of specific examples. 
3. 	Apply the numerical techniques to a solved problem in astrodynamics to demonstrate proficiency.
<a id='Yang'></a>[For example: Yang, et al. “Systematic Direct Approach for Optimizing Continuous-thrust Earth-orbit Transfers.” Chinese Journal of Aeronautics. 22/1 Feb. 2009. http://www.sciencedirect.com/science/article/pii/S1000936108600692
    1. Reproduce travel time and trajectory in table 2 and fig. 5
     2. Reproduce optimal 12-burn LEO-GEO transfer (table 5, etc.)]
4. 	Apply the numerical techniques to an original problems in astrodynamics. 
[For example: Extension and application, using information from Sanchez Cuartielles, et al. “Gravitational capture opportunites for asteroid retrieval missions.” 63rd International Astronautical Congress, 2012. http://strathprints.strath.ac.uk/41127/3/Sanchez_JP_et_al_Pure_Gravitational_capture_opportunities_for_asteroid_retrieval_missions_Oct_2012.pdf
     1.Develop trajectory to use L1/L2 gateways to transfer from geocentric orbit to rendezvous with NEO
     2.Develop trajectory to bring NEO back through L1/L2 gateway to geocentric orbit]
Primary resource used will be MATLAB.

### MEETING SCHEDULE ###
* Student will commit 6-9 hour per week during semester
* Student will maintain a weekly written log of progress
* Voice contact with the instructor will be maintained as needed

### EXPECTED OUTCOME / PRODUCT ###
Document describing:
1. 	Fundamentals and application of the single shooting technique to boundary value problems with mixed boundary conditions without and with optimization
2. 	Fundamentals and application of the multiple shooting technique to boundary value problems with mixed boundary conditions without and with optimization
3. 	Application of shooting technique to solved problem in astrodynamics
4.	Application of shooting technique to a problem in astrodynamics

### METHOD OF EVALUATION###
Weekly progress reports
Final documentation of studies



<a id='WeeklyProg'></a>
## Weekly Progress##
[Top](#Top)
### Week 1: July 18-23###
#### Accomplished####
* Set up Jupyter Notebook to hold research, results, and code
    * VPython install now works, but you would need Jupyter to run the code, so I am using it for the markdown text and for links to programs hosted at www.glowscript.org
* Began to explore shooting method and orbital applications
    * Wrote single shooting script to deal with heat flow along a cooling fin: http://www.glowscript.org/#/user/jgates/folder/OrbitalMechanics/program/CoolingFinShooting
    * Wrote single shooting script to deal with orbital transfer between 200 km and 600 km (around Earth): http://www.glowscript.org/#/user/jgates/folder/OrbitalMechanics/program/EarthOrbitShooting1
    * Wrote single shooting script to find Moon Lagrange point $L_1$ in circular restricted 3-body problem (CR3BP): http://www.glowscript.org/#/user/jgates/folder/OrbitalMechanics/program/OrbitShootingLunarL1
    * Wrote single shooting script to find necessary $v_i$ for 200 km halo orbit around $L_1$ in CR3BP - not working yet: http://www.glowscript.org/#/user/jgates/folder/OrbitalMechanics/program/LunarL1HaloOrbitShooting
    * Script to investigate solution of halo orbit: http://www.glowscript.org/#/user/jgates/folder/OrbitalMechanics/program/LunarL1HaloOrbitInvestigation

#### Sources and References####
* Lecture slides on single and multiple shooting: http://www.mech.utah.edu/~pardyjak/me6700/Lect15_BoundEigenvalueProblemsCh27.pdf
* Circular Restricted 3-Body Problem: http://ccar.colorado.edu/geryon/CRTBP.html
* Lo and Ross, Conference paper on lunar $L_1$ gateway: http://www.gg.caltech.edu/~mwl/publications/papers/lunarGateway.pdf
* Koon, Calculating the Halo Orbit (Shooting): https://web.archive.org/web/20060916192723/https://www.cds.caltech.edu/help/uploads/wiki/files/39/lecture_halo_2004.pdf
* Powell, Project on traveling to $L_1$, manifolds, etc.: http://ccar.colorado.edu/asen5050/projects/projects_2014/Powell_Sara/
* Project on using shooting method to find halo orbit: http://ccar.colorado.edu/asen5050/projects/projects_2012/dinkelk/extension.html
* 

#### Time ####
* Approximately 12 hours

#### Questions and tasks remaining####
* Is the low-thrust continuous thrust maneuver described in [Yang](#Yang) radial, in a constant direction, or something else?
* How can I extend my shooting algorithm to multiple shooting? Setting the interim conditions seems OK, but how are the discontinuities reconciled?
    * Is the convergence issue with the halo orbit due to insufficient numerical accuracy in the preceding steps ($L_1$ accuracy, $v_i$ accuracy, for example), or a fundamental issue with my shooting process?
* Need to fix the CR3BP problem - have to move Earth off of the origin and the moon in a bit, leaving the origin as the barycenter. ...then recalculate 
* Need to replace binary search algorithm with a secant algorithm
    

In [1]:
from math import *
from __future__ import division, print_function
from vpython import *
import numpy as np

Tinf = 300 # temp of environment
T_0 = 500 # initial temp @ hot end of fin
T_1 = 400 # initial temp@ cold end of fin


L = 1
dx = .1

M2 = .1 # constant

gK = graph(background=color.white, foreground=color.black, y=0, x=600, width=600,height=200,
              title='Temp vs. location', xtitle='Location (m)', ytitle='Temp (K)')
Tvsx = gcurve(color=color.red)
TvsxDesired = gdots(color = color.blue)
TvsxDesired.plot(pos = (0,T_0))
TvsxDesired.plot(pos = (L,T_1))

gK2 = graph(background=color.white, foreground=color.black, y=200, x=600, width=600,height=200,
              title='End error vs. guess', xtitle='Guess (K/m)', ytitle='End Temp (K)')
mvsz = gcurve(color=color.red)
mvszdots = gdots(color=color.blue)

gK3 = graph(background=color.white, foreground=color.black, y=400, x=600, width=600,height=200,
              title='Guess evolution', xtitle='n', ytitle='dTdx Guess (K/m)')
guessdots = gdots(color=color.blue)
final= gcurve(color=color.red)


def getTemp(dTdxGuess):
    y1 = [T_0] # temp values along bar
    y2 = [dTdxGuess] # dTdx values along bar
    Tvsx.plot(pos=(0,T_0))
    for x in np.arange(dx,L+dx,dx):
        rate(100000)
        y1dot = -y2[-1]
        y2dot = M2*(y1[-1]-Tinf)

        y1.append(y1[-1] + y1dot*dx)
        y2.append(y2[-1] + y2dot*dx)

        Tvsx.plot(pos=(x,y1[-1]))
    return(y1[-1])
    

dTdx1 = 2 # low guess for dTdx at left end (92 is good)
dTdx2 = 1000# high guess for dTdx at left end (92 is good)

Tend1 = getTemp(dTdx1)
guessdots.plot(pos=(0,dTdx1))
Tend2 = getTemp(dTdx2)
guessdots.plot(pos=(1,dTdx2))
n = 2
 
ms = [Tend1,Tend2]
guesses = [dTdx1,dTdx2]

Tf = T_0

while abs(Tf - T_1) > 1e-6:
    rate(10000)
    newguess = (dTdx1+dTdx2)/2
    guesses.append(newguess)
    #print(newguess)

    Tf = getTemp(newguess)
    ms.append(Tf)
    
    if Tf-T_1>0: #both guesses too high
        dTdx1 = newguess
        Tend1 = Tf
    else:
        dTdx2 = newguess
        Tend2 = Tf
    
    mvsz.plot(pos=(newguess,Tf-T_1))
    mvszdots.plot(pos=(newguess,Tf-T_1))

    guessdots.plot(pos=(n,newguess))
    n = n + 1

dTdxfinal = newguess
final.plot(pos=(0,dTdxfinal))
final.plot(pos=(n-1,dTdxfinal))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>